**Creating the Connection with MySQL**

In [0]:

driver = "org.mariadb.jdbc.Driver"

database_host = "junction.proxy.rlwy.net"
database_port = "56441"
database_name = ""
user = ""
password = ""

url = f"jdbc:mysql://{database_host}:{database_port}/{database_name}"

# Lista de tabelas a serem carregadas
tabelas = ["deliveries", "drivers", "hubs", "payments", "stores"]

print(url)

jdbc:mysql://junction.proxy.rlwy.net:56441/delivery


In [0]:

# Consultar as tabelas existentes no banco de dados
query = "(SELECT table_name FROM information_schema.tables WHERE table_schema = 'delivery') as tables" 

# Carregar as tabelas como um DataFrame
tables_df = spark.read.format("jdbc") \
    .option("url", url) \
    .option("dbtable", query) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", driver) \
    .load()


display(tables_df)

TABLE_NAME
channels
deliveries
drivers
hubs
orders
payments
stores


In [0]:
# Criar o banco de dados, caso não exista
spark.sql("CREATE DATABASE IF NOT EXISTS deliveryfood")

DataFrame[]

In [0]:
spark.sql("DESCRIBE DATABASE EXTENDED deliveryfood").show(truncate=False)


+-------------------------+--------------------------------------------------------+
|database_description_item|database_description_value                              |
+-------------------------+--------------------------------------------------------+
|Catalog Name             |adb_rgdev                                               |
|Namespace Name           |deliveryfood                                            |
|Comment                  |                                                        |
|Location                 |                                                        |
|Owner                    |deltaanalytica@hotmail.com                              |
|Properties               |                                                        |
|Predictive Optimization  |ENABLE (inherited from METASTORE metastore_azure_eastus)|
+-------------------------+--------------------------------------------------------+



In [0]:
# Nome do banco de dados no Databricks para salvar as tabelas
databricks_database_name = "deliveryfood"

In [0]:

for table in tabelas:
    df = spark.read.format("jdbc") \
        .option("url", url) \
        .option("dbtable", table) \
        .option("user", user) \
        .option("password", password) \
        .option("driver", driver) \
        .load()
    
    df.write.format("delta").mode("overwrite").saveAsTable(f"{databricks_database_name}.{table}")

In [0]:
%sql
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'deliveryfood';


table_name
channels
payments
hubs
stores
drivers
deliveries
